In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score  #交叉验证评分
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error #均方根误差
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
# 导入数据，路径中要么用\\或/或者在路径前加r
dataset = pd.read_csv('../data/train_chuli.csv')

In [9]:
# 输出数据预览
print(dataset.head())

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  \
0            1         0       3    1  22.0      1      0   7.2500   
1            2         1       1    0  38.0      1      0  71.2833   
2            3         1       3    0  26.0      0      0   7.9250   
3            4         1       1    0  35.0      1      0  53.1000   
4            5         0       3    1  35.0      0      0   8.0500   

   family_size  Embarked_C  Embarked_Q  Embarked_S  
0            2           0           0           1  
1            2           1           0           0  
2            1           0           0           1  
3            2           0           0           1  
4            1           0           0           1  


In [10]:
from sklearn.model_selection import train_test_split #划分数据集
# 准备训练数据
x = dataset.drop(['Survived'], axis = 1)
y = dataset['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(623, 11) (268, 11) (623,) (268,)


In [11]:
# !pip install lightgbm
import lightgbm as lgb

In [12]:
from matplotlib.cm import rainbow 

#决策树
rf_scores = []

rf_classifier = lgb.LGBMClassifier(num_leaves=35,
                        learning_rate=0.05,
                        n_estimators=100)
rf_classifier.fit(x_train, y_train)

LGBMClassifier(learning_rate=0.05, num_leaves=35)

LGBMClassifier
- boosting_type：提升类型。默认值为 'gbdt'，还可以选择 'dart' 或 'goss'。'gbdt' 是普通的梯度提升算法，'dart' 在 'gbdt' 的基础上引入了 dropout 技术，'goss' 是一种基于梯度的单边采样算法。
- num_leaves：叶子节点数。默认值为 31，它是模型复杂度的一个重要参数。增加 num_leaves 可以提高模型的复杂度，但也会增加过拟合的风险。
max_depth：树的最大深度。默认值为 -1，表示没有限制。增加 max_depth 可以提高模型的复杂度，但也会增加过拟合的风险。
learning_rate：学习率。默认值为 0.1，用于控制每次迭代时模型参数的更新步长。较小的学习率可以使模型更加稳定，但需要更多的迭代次数才能达到最优解。
n_estimators：弱学习器的数量。默认值为 100，表示使用 100 个基学习器进行训练。
objective：损失函数。默认值为 'binary'，可用于二分类和多分类问题。还可以选择其他损失函数，如 'multiclass'、'softmax'、'regression' 等。
colsample_bytree：列采样率。默认值为 1.0，表示使用所有的特征进行训练。较小的值可以减少过拟合的风险。
subsample：行采样率。默认值为 1.0，表示使用所有的样本进行训练。较小的值可以减少过拟合的风险。
reg_alpha：L1 正则化系数。默认值为 0.0，表示不进行 L1 正则化。较大的值可以增加 L1 正则化的强度，减少过拟合的风险。
reg_lambda：L2 正则化系数。默认值为 0.0，表示不进行 L2 正则化。较大的值可以增加 L2 正则化的强度，减少过拟合的风险。

In [13]:
#评价指标
import numpy as np

# 计算混淆矩阵
def compute_confusion_matrix(precited,expected):
    part = precited ^ expected             # 对结果进行分类，亦或使得判断正确的为0,判断错误的为1
    pcount = np.bincount(part)             # 分类结果统计，pcount[0]为0的个数，pcount[1]为1的个数
    tp_list = list(precited & expected)    # 将TP的计算结果转换为list
    fp_list = list(precited & ~expected)   # 将FP的计算结果转换为list
    tp = tp_list.count(1)                  # 统计TP的个数
    fp = fp_list.count(1)                  # 统计FP的个数
    tn = pcount[0] - tp                    # 统计TN的个数
    fn = pcount[1] - fp                    # 统计FN的个数
    return tp, fp, tn, fn

# 计算常用指标
def compute_indexes(tp, fp, tn, fn):
    accuracy = (tp+tn) / (tp+tn+fp+fn)     # 准确率
    precision = tp / (tp+fp)               # 精确率
    recall = tp / (tp+fn)                  # 召回率
    F1 = (2*precision*recall) / (precision+recall)    # F1
    return accuracy, precision, recall, F1


In [14]:
rfc_score = rf_classifier.score(x_test, y_test)#测试集准确率,j即accuracy
expected = y_test.astype('int32')#预期值
# print(type(expected))
# print(expected)
rfc_pre=rf_classifier.predict(x_test)
precited = rfc_pre.astype('int32')#模型预测值
# print(type(precited))
tp, fp, tn, fn = compute_confusion_matrix(precited, expected)
accuracy, precision, recall, F1 = compute_indexes(tp, fp, tn, fn)
rfc_pre = rf_classifier.predict(x_test)
rfc_auc = metrics.roc_auc_score(y_test, rfc_pre)
rfc_rmse = np.sqrt(mean_squared_error(y_test, rfc_pre))
print('TP FP TN FN')
print('%d %d %d %d'%(tp,fp,tn,fn))
print('accuracy precision recall F1')
print('%.4lf %.4lf %.4lf %.4lf'%(accuracy, precision, recall, F1))
print('auc rmse')
print('%.4lf %.4lf'%(rfc_auc, rfc_rmse))

TP FP TN FN
71 17 151 29
accuracy precision recall F1
0.8284 0.8068 0.7100 0.7553
auc rmse
0.8044 0.4143
